In [5]:
import pandas as pd

df = pd.read_csv('TEST_ML_BIG.csv')

In [ ]:
import requests

def get_make_model_id(year, make, model):
    url = f"https://carapi.app/api/trims?year={year}&make={make}&model={model}"
    response = requests.get(url)
    
    # 請求成功
    if response.status_code == 200:
        trims_data = response.json()
        if trims_data['collection']['count'] > 0:
            make_model_id = trims_data['data'][0]['make_model_id']
            return make_model_id, trims_data
    return None, None

def get_trim_id(make_model_id, trims_data, model_description):
    for trim in trims_data['data']:
        if model_description in trim['description']:
            return trim['id']
    return None

def get_weight(make_model_id, trim_id, year, make, model, weight_type):
    url = f"https://carapi.app/api/bodies?make_model_id={make_model_id}&year={year}&make={make}&model={model}"
    
    # 只有在 trim_id 不为 None 时才添加 make_model_trim_id 参数
    if trim_id is not None:
        url += f"&make_model_trim_id={trim_id}"

    response = requests.get(url)
    
    if response.status_code == 200:
        bodies_data = response.json()
        for body in bodies_data['data']:
            if body.get(weight_type) is not None:
                return body[weight_type]
    return None


# 在 DataFrame 中新增 'weight' 列
df['weight'] = None

# 遍历 DataFrame 的每一行
for index, row in df.iterrows():
    print(index, end=" ")
    year = row['Model Year']
    make = row['Make']
    model = row['Model']

    # 从 Model 中提取关键字
    keywords = model.split()[0:]

    # 通过第一个 URL 获取 make_model_id，并同时返回 trims_data
    make_model_id, trims_data = get_make_model_id(year, make, keywords[0])
    if make_model_id is not None:
        # 获取 trim_id
        for i in keywords:
            trim_id = get_trim_id(make_model_id, trims_data, i)

        # 通过第二个 URL 获取 gross_weight 或 curb_weight
        weight = get_weight(make_model_id, trim_id, year, make, keywords[0], "curb_weight")
        if weight is None:
            # 如果 gross_weight 找不到，则none
            weight = None

        # 将 weight 填充到 DataFrame 中
        print(weight)
        df.at[index, 'weight'] = weight

# 打印更新后的 DataFrame
print(df)


0 1 2 3 6069
4 6069
5 5445
6 5445
7 5445
8 5644
9 5644
10 11 None
12 None
13 None
14 5529
15 4773
16 4773
17 5137
18 3895
19 None
20 3638
21 3638
22 3638
23 3638
24 None
25 None
26 None
27 None
28 3858
29 3858
30 3858
31 3858
32 3957
33 3957
34 4750
35 4238
36 3948
37 4123
38 4123
39 5785
40 5785
41 3351
42 3351
43 3351
44 3351
45 3535
46 6542
47 6542
48 6542
49 6542
50 6542
51 6542
52 5616
53 5616
54 5473
55 5473
56 3858
57 3858
58 3858
59 3858
60 3957
61 6528
62 6528
63 6528
64 6528
65 5490
66 5490
67 5490
68 5490
69 5346
70 4013
71 3858
72 3858
73 3957
74 4750
75 4238
76 4238
77 6340
78 5346
79 5346
80 5346
81 5346
82 5346
83 5346
84 5730
85 5678
86 5663
87 6542
88 6542
89 6542
90 6542
91 6542
92 6542
93 6542
94 6542
95 6542
96 6542
97 6542
98 6542
99 6542
100 6542
101 6542
102 6542
103 6542
104 5616
105 5616
106 5616
107 5616
108 5473
109 5473
110 5473
111 5473
112 6528
113 6528
114 6528
115 6528
116 6528
117 6528
118 6528
119 6528
120 6528
121 6528
122 6528
123 6528
124 6528
125 6

In [ ]:
# 将 DataFrame 导出为 Excel 文件
df.to_excel('curb_weight_bigdata.xlsx', index=False)